<a href="https://colab.research.google.com/github/khanhsvan/Flight_booking_system/blob/main/RandomForest_SVM%20tuy%E1%BA%BFn%20t%C3%ADnh(Ho%C3%A0n%20Thi%E1%BB%87n).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Đọc dữ liệu từ các file CSV


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Đọc dữ liệu từ các file CSV và JSON
games_df = pd.read_csv('games.csv')
recommendations_df = pd.read_csv('recommendations.csv', error_bad_lines=False)
users_df = pd.read_csv('users.csv', error_bad_lines=False)
# metadata_df = pd.read_json('games_metadata.json', lines=True)

# Loại bỏ các dòng có dữ liệu thiếu trong 'recommendations_df'
recommendations_df.dropna(inplace=True)
games_df.dropna(inplace=True)
users_df.dropna(inplace=True)

# Chuyển đổi cột 'date_release' của DataFrame games_df thành định dạng datetime
games_df['date_release'] = pd.to_datetime(games_df['date_release'])

# Lọc các dòng từ năm 2020 trở đi
games_df = games_df[games_df['date_release'] > '2020-01-01']




<ipython-input-1-245689d590ad>:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  recommendations_df = pd.read_csv('recommendations.csv', error_bad_lines=False)
Skipping line 128545: expected 8 fields, saw 14

Skipping line 149972: expected 8 fields, saw 9

Skipping line 235663: expected 8 fields, saw 11

Skipping line 278516: expected 8 fields, saw 11
Skipping line 299946: expected 8 fields, saw 9

Skipping line 407070: expected 8 fields, saw 11
Skipping line 428497: expected 8 fields, saw 12

Skipping line 471341: expected 8 fields, saw 11

Skipping line 557035: expected 8 fields, saw 11
Skipping line 578457: expected 8 fields, saw 9

Skipping line 642731: expected 8 fields, saw 9

Skipping line 685574: expected 8 fields, saw 9

Skipping line 728432: expected 8 fields, saw 10
Skipping line 771271: expected 8 fields, saw 11

Skipping line 856973: expected 8 fields, saw 15

Skipping line 92123

In [2]:
# Tạo mô hình học máy cộng tác
# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
train, test = train_test_split(recommendations_df, test_size=0.6, random_state=42)



In [3]:
# Mô hình học máy cộng tác với số cây giảm xuống
rf_model = RandomForestRegressor(n_estimators=40, random_state=21)
rf_model.fit(train[['user_id', 'app_id']], train['hours'])


RandomForestRegressor(n_estimators=40, random_state=21)

In [4]:
# Đánh giá mô hình
predictions = rf_model.predict(test[['user_id', 'app_id']])
rmse = mean_squared_error(test['hours'], predictions, squared=False)
print(f'Collaborative Filtering RMSE: {rmse}')


Collaborative Filtering RMSE: 215.30905711017581


In [5]:
# Tạo mô hình dựa trên nội dung
# Sử dụng TF-IDF để vectơ hóa mô tả game
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(games_df['user_reviews'].values.astype('U'))


In [6]:
# Tính toán ma trận tương đồng cosine giữa các game
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [7]:
# Hàm để đề xuất game dựa trên nội dung
def content_based_recommendation(game_id):
    idx = games_df[games_df['app_id'] == game_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    game_indices = [i[0] for i in sim_scores]
    return games_df['app_id'].iloc[game_indices]


In [8]:
# Tạo mô hình học máy tuyến tính
linear_model = LinearRegression()
linear_model.fit(train[['user_id', 'app_id']], train['hours'])


LinearRegression()

In [9]:
# Đánh giá mô hình tuyến tính
linear_predictions = linear_model.predict(test[['user_id', 'app_id']])
linear_rmse = mean_squared_error(test['hours'], linear_predictions, squared=False)
print(f'Linear Regression RMSE: {linear_rmse}')


Linear Regression RMSE: 228.40899328750493


In [18]:
from itertools import product

# Đề xuất game dựa trên giờ chơi từ mô hình tuyến tính cho từng người dùng
def linear_based_recommendation(user_ids, top_n=5):
    # Tạo một dataframe chứa thông tin của người dùng và các trò chơi
    user_games = games_df[games_df['app_id'].isin(recommendations_df['app_id'])]

    # Tạo một dataframe mới với tất cả các cặp user_id và app_id có thể
    user_app_combinations = pd.DataFrame(list(product(user_ids, user_games['app_id'])), columns=['user_id', 'app_id'])

    # Dự đoán giờ chơi từ mô hình tuyến tính
    user_app_combinations['predicted_hours'] = linear_model.predict(user_app_combinations[['user_id', 'app_id']])

    # Sắp xếp theo user_id và predicted_hours
    user_app_combinations.sort_values(by=['user_id', 'predicted_hours'], ascending=[True, False], inplace=True)

    # Lấy top N game được đề xuất cho mỗi người dùng
    top_n_games_per_user = user_app_combinations.groupby('user_id').head(top_n)

    return top_n_games_per_user

# Chọn một danh sách user_ids để đề xuất
user_ids_to_recommend = [7089523, 13509147, 8443010]  # Thay đổi user_ids tùy ý

# Đề xuất game dựa trên giờ chơi cho từng người dùng
recommended_games_linear = linear_based_recommendation(user_ids_to_recommend)
print(f'Linear Regression Recommendations for Users:\n{recommended_games_linear}')
# Merge dataframe của game và recommendation để có thông tin về tên game
recommended_games_with_names = pd.merge(recommended_games_linear, games_df[['app_id', 'title']], on='app_id', how='left')

# In ra dataframe kết quả
print(recommended_games_with_names)

Linear Regression Recommendations for Users:
      user_id  app_id  predicted_hours
65    7089523  261550       245.053967
63    7089523  368260       232.004147
75    7089523  397540       228.423426
70    7089523  493520       216.685803
76    7089523  495420       216.453448
297   8443010  261550       246.916122
295   8443010  368260       233.866302
307   8443010  397540       230.285581
302   8443010  493520       218.547958
308   8443010  495420       218.315603
181  13509147  261550       253.886216
179  13509147  368260       240.836397
191  13509147  397540       237.255676
186  13509147  493520       225.518053
192  13509147  495420       225.285698
     user_id  app_id  predicted_hours                                 title
0    7089523  261550       245.053967          Mount & Blade II: Bannerlord
1    7089523  368260       232.004147                Marvel's Midnight Suns
2    7089523  397540       228.423426                         Borderlands 3
3    7089523  493520       